<a href="https://colab.research.google.com/github/githublior/Monet_cyclegan/blob/main/inference_p2m.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

INFERENCE NOTEBOOK Picture To Monet Cycle Gan

Student:
Lior shimon 341348498 lior.cbh@gmail.com,
Nadav Feige 205870421 nadavfeige@gmail.com

In [1]:
 from google.colab import drive
 
drive.mount('/content/drive', force_remount=True)
 
# path = "/content/drive/MyDrive/temp/data"
path = "/content/drive/MyDrive/Colab Notebooks/deep learning Blutman (CGAN)/data"

Mounted at /content/drive


In [2]:
import torch
import random, torch, os, numpy as np
import torch.nn as nn
import torch.nn.functional as F
import copy
from PIL import Image
import os
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torch.optim as optim
import numpy as np
import torchvision.models
from torchvision.utils import save_image
import torchvision.transforms
import random
import time
import sys
from tqdm import tqdm
from keras.preprocessing.image import ImageDataGenerator
from skimage import io




In [3]:
#HYPER Parameters
 
# import torch
import albumentations as A
# from albumentations.pytorch import ToTensorV2
 
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
pre_dir  = path + "/301_pic"
pre_m_dir = path + "/aug_monet"
TRAIN_DIR = path +"/data/train"
VAL_DIR = path + "/data/dev"
BATCH_SIZE = 2
LEARNING_RATE_G = 2e-4
LEARNING_RATE_D = 2e-4
LAMBDA_IDENTITY = 5
LAMBDA_CYCLE = 0.5
NUM_WORKERS = 1
WD =  1e-4
OLD = 2#17 for saved_images
NUM_EPOCHS = 45 - OLD
LOAD_MODEL = True
SAVE_MODEL = True
PRE_G = False 

CHECKPOINT_GEN_U = path+"/genu.pth.tar"
 
CHECKPOINT_GEN_P = path+"/genp.pth.tar"
CHECKPOINT_GEN_M = path+"/genm.pth.tar"
 
CHECKPOINT_CRITIC_P = path+"/criticp.pth.tar"
CHECKPOINT_CRITIC_M = path+"/criticm.pth.tar"
 
# data augmentation
transforms = A.Compose(
    [
        # A.Resize(width=256, height=256),
        # A.HorizontalFlip(p=0.5),
        # A.RandomSizedCrop([180, 230], 256, 256, w2h_ratio=1.0, p = 0.7),
        A.Normalize(mean=[0, 0, 0], std=[0.5, 0.5, 0.5], max_pixel_value=255),
        # ToTensor(),
    ],
    # additional_targets={"image0": "image"},
)

In [4]:
#Useful functions.

def save_checkpoint(model, optimizer, filename= path+"/my_checkpoint.pth.tar"):
    print("=> Saving checkpoint")
    checkpoint = {
        "state_dict": model.state_dict(),
        "optimizer": optimizer.state_dict(),
    }
    torch.save(checkpoint, filename)
    


def load_checkpoint(checkpoint_file, model, optimizer, lr):
    print("=> Loading checkpoint")
    checkpoint = torch.load(checkpoint_file, map_location=DEVICE)
    model.load_state_dict(checkpoint["state_dict"])
    optimizer.load_state_dict(checkpoint["optimizer"])

    # If we don't do this then it will just have learning rate of old checkpoint
    # and it will lead to many hours of debugging \:
    for param_group in optimizer.param_groups:
        param_group["lr"] = lr


def seed_everything(seed=12):
    os.environ["PYTHONHASHSEED"] = str(seed)
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

def gen_monet():
  datagen = ImageDataGenerator( 
          featurewise_center = True,
          featurewise_std_normalization = True,
          shear_range = 0.2,
          horizontal_flip = True,
          vertical_flip = True,
          brightness_range = (0.5, 1.5))

  image_directory =  path + '/30_monet' #r'C:Users---train/'
  SIZE = 256
  dataset = []
  my_images = os.listdir(image_directory)
  for i, image_name in enumerate(my_images):    
      if (image_name.split('.')[1] == 'jpg'):        
          image = io.imread(image_directory + '/'+image_name)        
          image = Image.fromarray(image, 'RGB')        
          image = image.resize((SIZE,SIZE)) 
          dataset.append(np.array(image))
  x = np.array(dataset)
  i = 0
  for batch in datagen.flow(x, batch_size=1,
                            save_to_dir= path + '/aug_monet' ,save_prefix='dr',save_format='jpg'):    
      i += 1    
      if i > 300:        
        print('here')
        break
  # r'C:Users---Augmented-images',

def select_some_pic():

  image_directory =  path + '/photo_jpg' #r'C:Users---train/'
  output_dir1 = path + "/300_pic"
  output_dir2 = path + "/301_pic"

  # random.shuffle(image_directory)
  my_images = os.listdir(image_directory)
  for i, filename in enumerate(my_images):   
    img = Image.open(os.path.join(image_directory, filename)) # images are color images
    # print(i)
    if i <300 : img.save(output_dir1+'/'+filename) 
    elif i <600: img.save(output_dir2+'/'+filename) 
    else: break 
      # if (image_name.split('.')[1] == 'jpg'):    
      # print(i)

def Average(lst):
    return sum(lst) / len(lst)

## takes in a module and applies the specified weight initialization
def weights_init_normal(m):
    '''Takes in a module and initializes all linear layers with weight
        values taken from a normal distribution.'''

    classname = m.__class__.__name__
    # for every Linear layer in a model
    if classname.find('Linear') != -1:
        y = m.in_features
    # m.weight.data shoud be taken from a normal distribution
        m.weight.data.normal_(0.0,0.02)#/np.sqrt(y))
    # m.bias.data should be 0
        m.bias.data.fill_(0)

In [5]:
# Generator Unet from colab
#https://colab.research.google.com/github/usuyama/pytorch-unet/blob/master/pytorch_unet_resnet18_colab.ipynb#scrollTo=b8EJl0hcC5DH

def convrelu(in_channels, out_channels, kernel, padding):
  return nn.Sequential(
    nn.Conv2d(in_channels, out_channels, kernel, padding=padding),
    nn.InstanceNorm2d(out_channels),
    nn.ReLU(inplace=True),
  )


class ResNetUNet(nn.Module):
  def __init__(self, n_class):
    super().__init__()

    self.base_model = torchvision.models.resnet18(pretrained=False)
    self.base_layers = list(self.base_model.children())

    self.layer0 = nn.Sequential(*self.base_layers[:3]) # size=(N, 64, x.H/2, x.W/2)
    self.layer0_1x1 = convrelu(64, 64, 1, 0)
    self.layer1 = nn.Sequential(*self.base_layers[3:5]) # size=(N, 64, x.H/4, x.W/4)
    self.layer1_1x1 = convrelu(64, 64, 1, 0)
    self.layer2 = self.base_layers[5]  # size=(N, 128, x.H/8, x.W/8)
    self.layer2_1x1 = convrelu(128, 128, 1, 0)
    self.layer3 = self.base_layers[6]  # size=(N, 256, x.H/16, x.W/16)
    self.layer3_1x1 = convrelu(256, 256, 1, 0)
    self.layer4 = self.base_layers[7]  # size=(N, 512, x.H/32, x.W/32)
    self.layer4_1x1 = convrelu(512, 512, 1, 0)

    self.upsample = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)

    self.conv_up3 = convrelu(256 + 512, 512, 3, 1)
    self.conv_up2 = convrelu(128 + 512, 256, 3, 1)
    self.conv_up1 = convrelu(64 + 256, 256, 3, 1)
    self.conv_up0 = convrelu(64 + 256, 128, 3, 1)

    self.conv_original_size0 = convrelu(3, 64, 3, 1)
    self.conv_original_size1 = convrelu(64, 64, 3, 1)
    self.conv_original_size2 = convrelu(64 + 128, 64, 3, 1)

    self.conv_last = nn.Conv2d(64, n_class, 1)

  def forward(self, input):
    x_original = self.conv_original_size0(input)
    x_original = self.conv_original_size1(x_original)

    layer0 = self.layer0(input)
    layer1 = self.layer1(layer0)
    layer2 = self.layer2(layer1)
    layer3 = self.layer3(layer2)
    layer4 = self.layer4(layer3)

    layer4 = self.layer4_1x1(layer4)
    x = self.upsample(layer4)
    layer3 = self.layer3_1x1(layer3)
    x = torch.cat([x, layer3], dim=1)
    x = self.conv_up3(x)

    x = self.upsample(x)
    layer2 = self.layer2_1x1(layer2)
    x = torch.cat([x, layer2], dim=1)
    x = self.conv_up2(x)

    x = self.upsample(x)
    layer1 = self.layer1_1x1(layer1)
    x = torch.cat([x, layer1], dim=1)
    x = self.conv_up1(x)

    x = self.upsample(x)
    layer0 = self.layer0_1x1(layer0)
    x = torch.cat([x, layer0], dim=1)
    x = self.conv_up0(x)

    x = self.upsample(x)
    x = torch.cat([x, x_original], dim=1)
    x = self.conv_original_size2(x)

    out = self.conv_last(x)

    return out

In [6]:
#dataset for inference

class PictureDataset(Dataset):
    def __init__(self, root_monet, transform):
        self.root_monet = root_monet
        self.monet_images = os.listdir(root_monet)
        self.monet_len = len(self.monet_images)
        self.transform = transform

    def __len__(self):
        return self.monet_len

    def __getitem__(self, index):
        monet_img = self.monet_images[index % self.monet_len]
        monet_path = os.path.join(self.root_monet, monet_img)
        monet_img = np.array(Image.open(monet_path))#.convert("RGB"))

        augmentations = self.transform(image=monet_img)
        monet_img = augmentations["image"]

        monet_img =  np.moveaxis(monet_img, -1,0).astype(float)  # change dimension shape : from  [256,256,3] to [3,256,256] to
        return monet_img


In [7]:
############## ###     Inference      #######################

def monet_generator(picture, gen_M):
  picture = picture.to(DEVICE)
  gen_M.eval()
  fake_monet = gen_M.forward(picture)
  return fake_monet



def create_sample_output(photo_ds, gen_M):#  d_scaler_M,d_scaler_P, g_scaler_M,g_scaler_P, j):
  print("creating fake monet from the dataset...")
  # loop = tqdm(photo_ds, leave=True)
  i=0
  for idx, picture in enumerate(photo_ds):
    i+=1
    print(i, '/', len(photo_ds))
    fake_monet = monet_generator(picture, gen_M)
    save_image(fake_monet , path+f"/images/" + str(i) + ".jpg", normalize=True)
    torch.cuda.empty_cache()

 
def main():

    model_1 = ResNetUNet(3)
    weights_init_normal(model_1)
    gen_M = model_1.double().to(DEVICE)
    opt_gen_M = optim.Adam( list(gen_M.parameters()),lr=LEARNING_RATE_G,betas = (0.5, .999))
 
    load_checkpoint(CHECKPOINT_GEN_M, gen_M, opt_gen_M, LEARNING_RATE_G )
 
    Picture_Path =  path + '/photo_jpg'
    dataset = PictureDataset(Picture_Path, transform=transforms)
    loader = DataLoader(dataset)
    create_sample_output(loader, gen_M)

 
if __name__ == "__main__":
    torch.cuda.empty_cache()
    seed_everything()
  
    main()
 
    print('end')

=> Loading checkpoint
creating fake monet from the dataset...
1 / 7040


/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


Le flux de sortie a été tronqué et ne contient que les 5000 dernières lignes.
2042 / 7040
2043 / 7040
2044 / 7040
2045 / 7040
2046 / 7040
2047 / 7040
2048 / 7040
2049 / 7040
2050 / 7040
2051 / 7040
2052 / 7040
2053 / 7040
2054 / 7040
2055 / 7040
2056 / 7040
2057 / 7040
2058 / 7040
2059 / 7040
2060 / 7040
2061 / 7040
2062 / 7040
2063 / 7040
2064 / 7040
2065 / 7040
2066 / 7040
2067 / 7040
2068 / 7040
2069 / 7040
2070 / 7040
2071 / 7040
2072 / 7040
2073 / 7040
2074 / 7040
2075 / 7040
2076 / 7040
2077 / 7040
2078 / 7040
2079 / 7040
2080 / 7040
2081 / 7040
2082 / 7040
2083 / 7040
2084 / 7040
2085 / 7040
2086 / 7040
2087 / 7040
2088 / 7040
2089 / 7040
2090 / 7040
2091 / 7040
2092 / 7040
2093 / 7040
2094 / 7040
2095 / 7040
2096 / 7040
2097 / 7040
2098 / 7040
2099 / 7040
2100 / 7040
2101 / 7040
2102 / 7040
2103 / 7040
2104 / 7040
2105 / 7040
2106 / 7040
2107 / 7040
2108 / 7040
2109 / 7040
2110 / 7040
2111 / 7040
2112 / 7040
2113 / 7040
2114 / 7040
2115 / 7040
2116 / 7040
2117 / 7040
2118 / 704